### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [ ]:
   import matplotlib.pyplot as plt
   import numpy as np
   import pandas as pd
   import math
   from collections import Counter
   from sklearn.datasets import load_wine
   from sklearn.ensemble import RandomForestClassifier
   from sklearn.metrics import accuracy_score, f1_score
   from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
   from sklearn.tree import DecisionTreeClassifier
   from sklearn.grid_search import GridSearchCV

   from functools import reduce
   from scipy.stats import randint as randint
   from scipy.stats import uniform

   try:
       from sklearn.model_selection import GridSearchCV
       from sklearn.model_selection import RandomizedSearchCV
       from sklearn.model_selection import StratifiedKFold
   except ImportError:
       from sklearn.cross_validation import GridSearchCV
       from sklearn.cross_validation import RandomizedSearchCV
       from sklearn.cross_validation import StratifiedKFold

   RND_SEED = 123


In [ ]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=0.95, criterion='gini', max_features='sqrt'):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.imperfection = self.__gini_imperfection
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.imperfection = self.__entropy_imperfection
        elif criterion == 'misclass':
        self.G_function = self.__misclass
            self.imperfection = self.__misclass_imperfection
        else:
            print('invalid criterion name')
            raise
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini_imperfection(self, y):
        s = y.shape[0]
        if (s == 0):
            s += 1
        cnt = Counter(y)
        m = np.array([cnt[i] for i in list(cnt)])
        return 1 - np.sum(m**2 / s**2)

    def __entropy_imperfection(self, y):
        s = y.shape[0]
        if (s == 0):
            s += 1
        cnt = Counter(y)
        m = np.array([cnt[i] for i in list(cnt)])
        return np.sum((m / s) * np.log2(m / s)) * (-1)

    def __misclass_imperfection(self, y):
        if (s == 0):
            s += 1
        cnt = Counter(y)
        m = np.array([cnt[i] for i in list(cnt)])
        return 1 - np.max(m / s)

    def __gini(self, l_c, l_s, r_c, r_s):
        r_s[r_s == 0] = 1
        l_s[l_s == 0] = 1
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (1 - ((np.array((l_c / l_s) ** 2).sum(axis=1)) * ((l_s / (l_s + r_s)).T) + (np.array((r_c / r_s) ** 2).sum(axis=1)) *((r_s / (l_s + r_s)).T)).T)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_c[l_c == 0] = 1
        l_s[l_s == 0] = 1
        r_c[r_c == 0] = 1
        r_s[r_s == 0] = 1
        return ((-np.array((l_c / l_s) * (np.log(l_c) - np.log(l_s)))).sum(axis=1)) - (np.array(((r_c / r_s) * (np.log(r_c) - np.log(r_s)))).sum(axis=1))

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s[l_s == 0] = 1
        r_s[r_s == 0] = 1
        return 1 - (l_c / l_s + r_c / r_s).max(axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[range(max(1, int(math.sqrt(n_feature))))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[range(max(1, int(math.log2(n_feature))))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(max(1, int(n_feature)))

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)
        if cut_size == 0:
            split_sorted_y = sorted_y
        else:
            split_sorted_y = sorted_y[cut_size:-cut_size]
        border = np.where(split_sorted_y[:-1] != split_sorted_y[1:])[0] + (cut_size + 1)
        if len(border) == 0:
            return np.inf, None
        ee = border - np.append(np.array([cut_size]), border[:-1])
        unitary = np.zeros((border.shape[0], class_number))
        unitary[np.arange(border.shape[0]), sorted_y[border - 1]] = 1
        class_increments = unitary * ee.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:cut_size], minlength=class_number)
        lcc = np.cumsum(class_increments, axis=0)
        rcc = np.bincount(sorted_y, minlength=class_number) - lcc
        l_sizes = border.reshape(lcc.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        g = self.G_function(lcc, l_sizes, rcc, r_sizes)
        le = l_sizes[np.argmin(g)][0]
        return g[np.argmin(g)], (sorted_x[le-1] + sorted_x[le]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        node = []
        if((not(depth is None) and depth == self.max_depth) or y.shape[0] <= self.min_samples_split or (Counter(y).most_common(1)[0][1] / y.shape[0]) >= self.sufficient_share):
            node.append(self.__class__.LEAF_TYPE)
            if(y.shape[0] > 0):
                node.append(Counter(y).most_common(1)[0][0])
            self.tree[node_id] = (node)
            return self.imperfection(y)
        node.append(self.__class__.NON_LEAF_TYPE)
        feature = self.get_feature_ids(x.shape[1])
        feature_threshold = np.array(list(map(lambda i: list(self.__find_threshold(x[:, i], y)), feature)))
        best = np.argmin(feature_threshold[:][0])
        node.append(feature[best])
        node.append(feature_threshold[best][1])
        self.tree[node_id] = (node)
        xl, xr, yl, yr = self.__div_samples(x, y, node[1], node[2])
        if(yl.shape[0] > 0 and yr.shape[0] > 0):
            self.feature_importances_[feature[best]] += self.imperfection(y)*(y.shape[0]) - self.__fit_node(xl, yl, 2*node_id+1, depth+1)*(yl.shape[0]) - self.__fit_node(xr, yr, 2*node_id+2, depth+1) * (yr.shape[0])
        else:
            node = []
            node.append(self.__class__.LEAF_TYPE)
            if(y.shape[0] > 0):
                node.append(Counter(y).most_common(1)[0][0])
            self.tree[node_id] = (node)
        return self.imperfection(y)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        fi = self.feature_importances_
        self.feature_importances_ = fi / np.sum(fi)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def score(self, X, y):
        pred = self.predict(X)
        sum = 0
        for i in range(y.shape[0]):
            sum += pred[i] == y[i]
        return sum / len(y)


In [ ]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [ ]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

## Проверка качества работы на wine

In [ ]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [ ]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

    df = pd.read_csv('./data/speed-dating-experiment/Speed Dating Data.csv', encoding='latin1')
    choseddf = df[['match','gender' ,'age_o', 'race_o','goal','race_o', 'round' ,'condtn',  'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha','dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',  'prob_o','met_o']].dropna()
    y = np.array(choseddf['match']).astype('int')
    X=np.array(choseddf[['gender' ,'age_o', 'race_o','goal','race_o', 'round' ,'condtn',  'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha','dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',  'prob_o','met_o']]).astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion='gini', max_features='sqrt')
    clf = DecisionTreeClassifier(min_samples_split=2,criterion='gini', max_features='sqrt')


## Проверка скорости работы на Speed Dating Data 

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

## Проверка качества работы на Speed Dating Data

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

## Задание 3

In [ ]:
   print(np.argsort(clf.feature_importances_)[-10:])


## Задание 4

In [ ]:
   print(np.argsort(my_clf.feature_importances_)[-10:])


## Задание 5

In [ ]:
   param_grid = {'n_estimators': range(1, 10),'max_depth': range(1, 50)}
   rfc = RandomForestClassifier(n_estimators=100, max_depth=2)
   Cv = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
   Cv.fit(X_train, y_train)
   print(Cv.best_params_)
